In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.inspection import PartialDependenceDisplay
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("HOTEL BOOKING CANCELLATION PREDICTION & RECOMMENDATION SYSTEM")
print("="*80)

# ============================================
# LOAD AND PREPROCESS DATA
# ============================================
df = pd.read_csv('Hotel Reservations.csv')
print(f"\nOriginal shape: {df.shape}")
df = df.drop('Booking_ID', axis=1)

# Fix invalid dates
mask_feb29_2018 = ((df['arrival_month'] == 2) & (df['arrival_date'] == 29) & (df['arrival_year'] == 2018))
if mask_feb29_2018.any():
    print(f"Correcting {mask_feb29_2018.sum()} invalid dates (2018-02-29 → 2018-02-28)")
    df.loc[mask_feb29_2018, 'arrival_date'] = 28

# Date feature engineering
df['arrival_datetime'] = pd.to_datetime(
    df['arrival_year'].astype(str) + '-' + 
    df['arrival_month'].astype(str).str.zfill(2) + '-' + 
    df['arrival_date'].astype(str).str.zfill(2), errors='coerce'
)
df['arrival_dayofweek'] = df['arrival_datetime'].dt.dayofweek
df['arrival_day'] = df['arrival_datetime'].dt.day
df['arrival_quarter'] = df['arrival_datetime'].dt.quarter
df['is_weekend_arrival'] = df['arrival_dayofweek'].isin([5, 6]).astype(int)
df = df.drop(['arrival_datetime', 'arrival_date'], axis=1)

print(f"After preprocessing: {len(df)} records")

# ============================================
# FEATURE DEFINITIONS
# ============================================
discrete_numerical_cols = [
    'no_of_adults', 'no_of_children', 'no_of_weekend_nights', 
    'no_of_week_nights', 'arrival_year', 'arrival_month',
    'arrival_dayofweek', 'arrival_day', 'arrival_quarter',
    'is_weekend_arrival', 'no_of_previous_cancellations', 
    'no_of_previous_bookings_not_canceled', 'no_of_special_requests'
]

continuous_numerical_cols = ['lead_time', 'avg_price_per_room']

numerical_cols = continuous_numerical_cols + discrete_numerical_cols

categorical_cols = [
    'type_of_meal_plan', 'room_type_reserved', 'market_segment_type', 
    'required_car_parking_space', 'repeated_guest'
]

actionable_features = {
    'avg_price_per_room': 'Offer 10-15% discount or complimentary room upgrade',
    'type_of_meal_plan': 'Offer discounted meal plan package',
    'room_type_reserved': 'Offer free room upgrade to premium category',
    'no_of_special_requests': 'Contact guest proactively to gather preferences',
    'required_car_parking_space': 'Offer complimentary parking',
}

# ============================================
# TRAIN-TEST SPLIT
# ============================================
train, test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['booking_status'])
print(f"\nTrain: {train.shape}, Test: {test.shape}")

x_train_orig = train.drop('booking_status', axis=1)
y_train = train['booking_status']
x_test_orig = test.drop('booking_status', axis=1)
y_test = test['booking_status']

# ============================================
# PREPROCESSING WITHOUT SCALING - ORIGINAL DATA ONLY
# ============================================
print("\n⚠️ Using ORIGINAL data (No scaling applied)")

preprocessor = ColumnTransformer(
    transformers=[
        ('encode', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), categorical_cols),
        ('passthrough', 'passthrough', numerical_cols)
    ]
)

x_train_processed = preprocessor.fit_transform(x_train_orig)
x_test_processed = preprocessor.transform(x_test_orig)

# Feature names
cat_names = list(preprocessor.named_transformers_['encode'].get_feature_names_out(categorical_cols))
all_feature_names = cat_names + numerical_cols

x_train_df = pd.DataFrame(x_train_processed, columns=all_feature_names)
x_test_df = pd.DataFrame(x_test_processed, columns=all_feature_names)

print(f"Feature columns: {len(all_feature_names)}")
print(f"  Categorical (encoded): {len(cat_names)}")
print(f"  Numerical (original): {len(numerical_cols)}")

# Encode target
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

print(f"\nTarget encoding: {dict(zip(le.classes_, le.transform(le.classes_)))}")
print(f"  0 = Canceled, 1 = Not_Canceled")

# SMOTE
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train_df, y_train_encoded)

print(f"\nAfter SMOTE: {len(x_train_resampled)} samples")
print(f"Class balance: {pd.Series(y_train_resampled).value_counts().to_dict()}")

# ============================================
# HELPER FUNCTION FOR COMPREHENSIVE METRICS
# ============================================
def calculate_metrics(y_true, y_pred, dataset_name="", model_name=""):
    """Calculate comprehensive classification metrics"""
    acc = accuracy_score(y_true, y_pred)
    
    # For binary classification, we focus on the positive class (0 = Canceled)
    precision = precision_score(y_true, y_pred, pos_label=0, zero_division=0)
    recall = recall_score(y_true, y_pred, pos_label=0, zero_division=0)
    f1 = f1_score(y_true, y_pred, pos_label=0, zero_division=0)
    
    return {
        'Model': model_name,
        'Dataset': dataset_name,
        'Accuracy': acc,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    }

def print_metrics_table(metrics_list):
    """Print metrics in a formatted table"""
    df_metrics = pd.DataFrame(metrics_list)
    print("\n" + df_metrics.to_string(index=False))
    return df_metrics

def plot_confusion_matrix(y_true, y_pred, title, filename):
    """Plot and save confusion matrix"""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Canceled', 'Not_Canceled'],
                yticklabels=['Canceled', 'Not_Canceled'])
    plt.title(title, fontsize=14, fontweight='bold')
    plt.ylabel('Actual', fontsize=12)
    plt.xlabel('Predicted', fontsize=12)
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"✓ Saved: {filename}")
    plt.close()

# ============================================
# APPROACH 1: STANDARD MODELS
# ============================================
print("\n" + "="*80)
print("APPROACH 1: STANDARD MODELS (No Clustering)")
print("="*80)

all_metrics = []

# Random Forest Standard
print("\n--- Random Forest (Standard) ---")
rf_standard = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=20, 
                                     min_samples_split=5, min_samples_leaf=2)
rf_standard.fit(x_train_resampled, y_train_resampled)

rf_train_pred = rf_standard.predict(x_train_resampled)
rf_test_pred = rf_standard.predict(x_test_df)

rf_train_metrics = calculate_metrics(y_train_resampled, rf_train_pred, "Train", "RF Standard")
rf_test_metrics = calculate_metrics(y_test_encoded, rf_test_pred, "Test", "RF Standard")

all_metrics.extend([rf_train_metrics, rf_test_metrics])

print(f"Train - Acc: {rf_train_metrics['Accuracy']:.4f}, Prec: {rf_train_metrics['Precision']:.4f}, Rec: {rf_train_metrics['Recall']:.4f}, F1: {rf_train_metrics['F1-Score']:.4f}")
print(f"Test  - Acc: {rf_test_metrics['Accuracy']:.4f}, Prec: {rf_test_metrics['Precision']:.4f}, Rec: {rf_test_metrics['Recall']:.4f}, F1: {rf_test_metrics['F1-Score']:.4f}")

plot_confusion_matrix(y_test_encoded, rf_test_pred, 
                     'Confusion Matrix - RF Standard (Test)', 
                     'confusion_matrix_rf_standard.png')

# XGBoost Standard
print("\n--- XGBoost (Standard) ---")
xgb_standard = XGBClassifier(n_estimators=100, random_state=42, max_depth=6, 
                             learning_rate=0.1, eval_metric='logloss')
xgb_standard.fit(x_train_resampled, y_train_resampled)

xgb_train_pred = xgb_standard.predict(x_train_resampled)
xgb_test_pred = xgb_standard.predict(x_test_df)

xgb_train_metrics = calculate_metrics(y_train_resampled, xgb_train_pred, "Train", "XGB Standard")
xgb_test_metrics = calculate_metrics(y_test_encoded, xgb_test_pred, "Test", "XGB Standard")

all_metrics.extend([xgb_train_metrics, xgb_test_metrics])

print(f"Train - Acc: {xgb_train_metrics['Accuracy']:.4f}, Prec: {xgb_train_metrics['Precision']:.4f}, Rec: {xgb_train_metrics['Recall']:.4f}, F1: {xgb_train_metrics['F1-Score']:.4f}")
print(f"Test  - Acc: {xgb_test_metrics['Accuracy']:.4f}, Prec: {xgb_test_metrics['Precision']:.4f}, Rec: {xgb_test_metrics['Recall']:.4f}, F1: {xgb_test_metrics['F1-Score']:.4f}")

plot_confusion_matrix(y_test_encoded, xgb_test_pred, 
                     'Confusion Matrix - XGB Standard (Test)', 
                     'confusion_matrix_xgb_standard.png')

# ============================================
# APPROACH 2: CLUSTER-BASED MODELS
# ============================================
print("\n" + "="*80)
print("APPROACH 2: CLUSTER-BASED MODELS")
print("="*80)

n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
train_clusters = kmeans.fit_predict(x_train_resampled)
test_clusters = kmeans.predict(x_test_df)

print(f"\nCluster Distribution:")
print(f"Train: {pd.Series(train_clusters).value_counts().sort_index().to_dict()}")
print(f"Test: {pd.Series(test_clusters).value_counts().sort_index().to_dict()}")

# Analyze clusters
for cid in range(n_clusters):
    mask = train_clusters == cid
    cancel_rate = (y_train_resampled[mask] == 0).mean()
    print(f"Cluster {cid}: Size={mask.sum()}, Cancellation Rate={cancel_rate:.2%}")

# Train separate models for each cluster
rf_cluster_models = {}
xgb_cluster_models = {}

for cid in range(n_clusters):
    print(f"\n--- Cluster {cid} ---")
    mask = train_clusters == cid
    x_cluster = x_train_resampled[mask]
    y_cluster = y_train_resampled[mask]
    
    # Random Forest
    rf_c = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=20,
                                  min_samples_split=5, min_samples_leaf=2)
    rf_c.fit(x_cluster, y_cluster)
    rf_cluster_models[cid] = rf_c
    
    rf_c_train_pred = rf_c.predict(x_cluster)
    rf_c_metrics = calculate_metrics(y_cluster, rf_c_train_pred, f"Train-C{cid}", "RF Cluster")
    all_metrics.append(rf_c_metrics)
    
    # XGBoost
    xgb_c = XGBClassifier(n_estimators=100, random_state=42, max_depth=6,
                         learning_rate=0.1, eval_metric='logloss')
    xgb_c.fit(x_cluster, y_cluster)
    xgb_cluster_models[cid] = xgb_c
    
    xgb_c_train_pred = xgb_c.predict(x_cluster)
    xgb_c_metrics = calculate_metrics(y_cluster, xgb_c_train_pred, f"Train-C{cid}", "XGB Cluster")
    all_metrics.append(xgb_c_metrics)
    
    print(f"RF  - Acc: {rf_c_metrics['Accuracy']:.4f}, Prec: {rf_c_metrics['Precision']:.4f}, Rec: {rf_c_metrics['Recall']:.4f}, F1: {rf_c_metrics['F1-Score']:.4f}")
    print(f"XGB - Acc: {xgb_c_metrics['Accuracy']:.4f}, Prec: {xgb_c_metrics['Precision']:.4f}, Rec: {xgb_c_metrics['Recall']:.4f}, F1: {xgb_c_metrics['F1-Score']:.4f}")

# Test cluster models - Overall
print("\n--- Cluster Models: Overall Test Performance ---")

rf_cluster_preds = np.array([rf_cluster_models[c].predict(x_test_df.iloc[[i]])[0] 
                             for i, c in enumerate(test_clusters)])
xgb_cluster_preds = np.array([xgb_cluster_models[c].predict(x_test_df.iloc[[i]])[0] 
                              for i, c in enumerate(test_clusters)])

rf_cluster_test_metrics = calculate_metrics(y_test_encoded, rf_cluster_preds, "Test", "RF Cluster")
xgb_cluster_test_metrics = calculate_metrics(y_test_encoded, xgb_cluster_preds, "Test", "XGB Cluster")

all_metrics.extend([rf_cluster_test_metrics, xgb_cluster_test_metrics])

print(f"RF  - Acc: {rf_cluster_test_metrics['Accuracy']:.4f}, Prec: {rf_cluster_test_metrics['Precision']:.4f}, Rec: {rf_cluster_test_metrics['Recall']:.4f}, F1: {rf_cluster_test_metrics['F1-Score']:.4f}")
print(f"XGB - Acc: {xgb_cluster_test_metrics['Accuracy']:.4f}, Prec: {xgb_cluster_test_metrics['Precision']:.4f}, Rec: {xgb_cluster_test_metrics['Recall']:.4f}, F1: {xgb_cluster_test_metrics['F1-Score']:.4f}")

plot_confusion_matrix(y_test_encoded, rf_cluster_preds, 
                     'Confusion Matrix - RF Cluster (Test)', 
                     'confusion_matrix_rf_cluster.png')

plot_confusion_matrix(y_test_encoded, xgb_cluster_preds, 
                     'Confusion Matrix - XGB Cluster (Test)', 
                     'confusion_matrix_xgb_cluster.png')

# Test accuracy per cluster
print("\n--- Per-Cluster Test Performance ---")
for cid in range(n_clusters):
    mask = test_clusters == cid
    
    rf_cluster_c_metrics = calculate_metrics(y_test_encoded[mask], rf_cluster_preds[mask], 
                                            f"Test-C{cid}", "RF Cluster")
    xgb_cluster_c_metrics = calculate_metrics(y_test_encoded[mask], xgb_cluster_preds[mask], 
                                             f"Test-C{cid}", "XGB Cluster")
    
    all_metrics.extend([rf_cluster_c_metrics, xgb_cluster_c_metrics])
    
    print(f"\nCluster {cid}:")
    print(f"RF  - Acc: {rf_cluster_c_metrics['Accuracy']:.4f}, Prec: {rf_cluster_c_metrics['Precision']:.4f}, Rec: {rf_cluster_c_metrics['Recall']:.4f}, F1: {rf_cluster_c_metrics['F1-Score']:.4f}")
    print(f"XGB - Acc: {xgb_cluster_c_metrics['Accuracy']:.4f}, Prec: {xgb_cluster_c_metrics['Precision']:.4f}, Rec: {xgb_cluster_c_metrics['Recall']:.4f}, F1: {xgb_cluster_c_metrics['F1-Score']:.4f}")

# ============================================
# COMPREHENSIVE METRICS SUMMARY
# ============================================
print("\n" + "="*80)
print("COMPREHENSIVE METRICS SUMMARY")
print("="*80)

metrics_df = print_metrics_table(all_metrics)

# Save metrics to CSV
metrics_df.to_csv('model_metrics_comprehensive.csv', index=False)
print("\n✓ Saved: model_metrics_comprehensive.csv")

# ============================================
# MODEL COMPARISON
# ============================================
print("\n" + "="*80)
print("MODEL COMPARISON (Test Set)")
print("="*80)

test_metrics_only = metrics_df[metrics_df['Dataset'].str.contains('Test') & 
                                ~metrics_df['Dataset'].str.contains('-C')]

print("\nTest Performance Comparison:")
print(test_metrics_only.to_string(index=False))

# Find best model by F1-Score (most balanced metric)
best_by_f1 = test_metrics_only.loc[test_metrics_only['F1-Score'].idxmax()]
best_by_acc = test_metrics_only.loc[test_metrics_only['Accuracy'].idxmax()]

print(f"\n🏆 BEST MODEL BY F1-SCORE: {best_by_f1['Model']}")
print(f"   Accuracy: {best_by_f1['Accuracy']:.4f}")
print(f"   Precision: {best_by_f1['Precision']:.4f}")
print(f"   Recall: {best_by_f1['Recall']:.4f}")
print(f"   F1-Score: {best_by_f1['F1-Score']:.4f}")

print(f"\n🎯 BEST MODEL BY ACCURACY: {best_by_acc['Model']}")
print(f"   Accuracy: {best_by_acc['Accuracy']:.4f}")
print(f"   Precision: {best_by_acc['Precision']:.4f}")
print(f"   Recall: {best_by_acc['Recall']:.4f}")
print(f"   F1-Score: {best_by_acc['F1-Score']:.4f}")

# Select best model
use_clustering = 'Cluster' in best_by_f1['Model']
if use_clustering:
    if 'RF' in best_by_f1['Model']:
        best_model_dict = rf_cluster_models
        model_type = 'RF-Cluster'
    else:
        best_model_dict = xgb_cluster_models
        model_type = 'XGB-Cluster'
    best_single_model = None
else:
    if 'RF' in best_by_f1['Model']:
        best_single_model = rf_standard
        model_type = 'RF-Standard'
    else:
        best_single_model = xgb_standard
        model_type = 'XGB-Standard'
    best_model_dict = None

# ============================================
# VISUALIZATION: METRICS COMPARISON
# ============================================
print("\n" + "="*80)
print("CREATING VISUALIZATION")
print("="*80)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
colors = ['#2E86AB', '#A23B72', '#F18F01', '#06A77D']

for idx, (metric, color) in enumerate(zip(metrics_to_plot, colors)):
    ax = axes[idx // 2, idx % 2]
    
    plot_data = test_metrics_only[['Model', metric]].sort_values(metric, ascending=False)
    
    bars = ax.barh(range(len(plot_data)), plot_data[metric], color=color, alpha=0.7, edgecolor='black')
    ax.set_yticks(range(len(plot_data)))
    ax.set_yticklabels(plot_data['Model'])
    ax.set_xlabel(metric, fontsize=12, fontweight='bold')
    ax.set_title(f'{metric} Comparison (Test Set)', fontsize=13, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for i, (bar, val) in enumerate(zip(bars, plot_data[metric])):
        ax.text(val + 0.005, i, f'{val:.4f}', va='center', fontsize=10)

plt.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('model_comparison_metrics.png', dpi=300, bbox_inches='tight')
print("✓ Saved: model_comparison_metrics.png")
plt.close()

# ============================================
# FEATURE IMPORTANCE
# ============================================
print("\n" + "="*80)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*80)

if use_clustering:
    importances = np.mean([m.feature_importances_ for m in best_model_dict.values()], axis=0)
else:
    importances = best_single_model.feature_importances_

feature_imp_df = pd.DataFrame({
    'feature': all_feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)

print("\nTop 15 Features (Overall):")
print(feature_imp_df.head(15).to_string(index=False))

# Overall feature importance plot
plt.figure(figsize=(10, 8))
top15 = feature_imp_df.head(15)
colors_feat = plt.cm.viridis(np.linspace(0.3, 0.9, len(top15)))
plt.barh(range(len(top15)), top15['importance'], color=colors_feat)
plt.yticks(range(len(top15)), top15['feature'])
plt.xlabel('Importance', fontsize=12)
plt.title(f'Feature Importance - {model_type} (Overall)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('feature_importance_overall.png', dpi=300, bbox_inches='tight')
print("✓ Saved: feature_importance_overall.png")
plt.close()

# ============================================
# FEATURE IMPORTANCE PER CLUSTER (if clustering used)
# ============================================
if use_clustering:
    print("\n" + "="*80)
    print("FEATURE IMPORTANCE BY CLUSTER")
    print("="*80)
    
    cluster_feature_importance = {}
    
    for cid in range(n_clusters):
        print(f"\n--- Cluster {cid} ---")
        model = best_model_dict[cid]
        
        cluster_imp_df = pd.DataFrame({
            'feature': all_feature_names,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        cluster_feature_importance[cid] = cluster_imp_df
        
        print(f"\nTop 15 Features:")
        print(cluster_imp_df.head(15).to_string(index=False))
        
        # Plot for each cluster
        plt.figure(figsize=(10, 8))
        top15_cluster = cluster_imp_df.head(15)
        colors_cluster = plt.cm.plasma(np.linspace(0.3, 0.9, len(top15_cluster)))
        plt.barh(range(len(top15_cluster)), top15_cluster['importance'], color=colors_cluster)
        plt.yticks(range(len(top15_cluster)), top15_cluster['feature'])
        plt.xlabel('Importance', fontsize=12)
        plt.title(f'Feature Importance - {model_type} - Cluster {cid}', fontsize=14, fontweight='bold')
        plt.gca().invert_yaxis()
        plt.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'feature_importance_cluster_{cid}.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: feature_importance_cluster_{cid}.png")
        plt.close()
    
    # Comparison plot: Side-by-side top 10 features for both clusters
    print("\n--- Creating Cluster Comparison Plot ---")
    fig, axes = plt.subplots(1, 2, figsize=(18, 8))
    
    for cid in range(n_clusters):
        ax = axes[cid]
        top10_cluster = cluster_feature_importance[cid].head(10)
        colors_comp = plt.cm.viridis(np.linspace(0.3, 0.9, len(top10_cluster)))
        
        ax.barh(range(len(top10_cluster)), top10_cluster['importance'], color=colors_comp)
        ax.set_yticks(range(len(top10_cluster)))
        ax.set_yticklabels(top10_cluster['feature'])
        ax.set_xlabel('Importance', fontsize=11)
        ax.set_title(f'Cluster {cid} - Top 10 Features', fontsize=13, fontweight='bold')
        ax.invert_yaxis()
        ax.grid(axis='x', alpha=0.3)
    
    plt.suptitle(f'Feature Importance Comparison Across Clusters', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig('feature_importance_clusters_comparison.png', dpi=300, bbox_inches='tight')
    print("✓ Saved: feature_importance_clusters_comparison.png")
    plt.close()

# ============================================
# PARTIAL DEPENDENCE PLOTS - TOP 4 FEATURES (OVERALL)
# ============================================
print("\n" + "="*80)
print("PARTIAL DEPENDENCE PLOTS (Top 4 Most Important Features - Overall)")
print("="*80)

# Get top 4 features (any type)
top_4_features = feature_imp_df.head(4)['feature'].tolist()

print(f"\nCreating PDPs for top 4 features: {top_4_features}")

for feat in top_4_features:
    print(f"\n--- Processing PDP for: {feat} ---")
    
    feat_idx = all_feature_names.index(feat)
    
    # Determine if feature is numerical or categorical (encoded)
    is_numerical = feat in numerical_cols
    
    if is_numerical:
        # Get original values from training data
        orig_values = x_train_orig[feat].values
        orig_min, orig_max = orig_values.min(), orig_values.max()
        orig_mean = orig_values.mean()
        orig_std = orig_values.std()
        
        print(f"  Type: Numerical")
        print(f"  Min: {orig_min:.2f}, Max: {orig_max:.2f}, Mean: {orig_mean:.2f}, Std: {orig_std:.2f}")
        
        is_continuous = feat in continuous_numerical_cols
        
        if is_continuous:
            # CONTINUOUS FEATURES - Comprehensive 4-panel visualization
            fig = plt.figure(figsize=(16, 12))
            gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)
            
            if use_clustering:
                model_for_pdp = list(best_model_dict.values())[0]
            else:
                model_for_pdp = best_single_model
            
            # 1. Standard PDP (top-left)
            ax1 = fig.add_subplot(gs[0, 0])
            display1 = PartialDependenceDisplay.from_estimator(
                model_for_pdp, x_train_resampled, features=[feat_idx],
                feature_names=all_feature_names, target=1, ax=ax1, grid_resolution=100
            )
            ax1.set_title(f'Partial Dependence Plot', fontsize=12, fontweight='bold')
            ax1.set_xlabel(f'{feat} (Original Scale)', fontsize=11, fontweight='bold')
            ax1.grid(alpha=0.3)
            
            # 2. PDP with confidence intervals (top-right)
            ax2 = fig.add_subplot(gs[0, 1])
            
            # Create range in original scale
            orig_range = np.linspace(orig_min, orig_max, 100)
            
            # Create synthetic data for PD calculation
            X_synthetic = np.tile(x_train_resampled.mean(axis=0), (100, 1))
            X_synthetic[:, feat_idx] = orig_range
            
            # Calculate predictions
            pd_values = model_for_pdp.predict_proba(X_synthetic)[:, 1]
            
            # Center the PD values
            pd_values_centered = pd_values - pd_values.mean()
            
            ax2.plot(orig_range, pd_values_centered, linewidth=2.5, color='#2E86AB', label='PDP')
            ax2.fill_between(orig_range, pd_values_centered, alpha=0.3, color='#2E86AB')
            ax2.set_xlabel(f'{feat} (Original Scale)', fontsize=11, fontweight='bold')
            ax2.set_ylabel('Partial Dependence (Centered)', fontsize=11, fontweight='bold')
            ax2.set_title(f'PDP with Shaded Area', fontsize=12, fontweight='bold')
            ax2.grid(alpha=0.3)
            ax2.axhline(y=0, color='red', linestyle='--', alpha=0.5, linewidth=1)
            
            # Add percentile markers
            percentiles = [10, 25, 50, 75, 90]
            for p in percentiles:
                val = np.percentile(orig_values, p)
                ax2.axvline(x=val, color='gray', linestyle=':', alpha=0.3, linewidth=1)
                ax2.text(val, ax2.get_ylim()[1]*0.95, f'P{p}', 
                        ha='center', va='top', fontsize=8, color='gray')
            
            # 3. Distribution of original values (bottom-left)
            ax3 = fig.add_subplot(gs[1, 0])
            ax3.hist(orig_values, bins=50, alpha=0.7, color='#A23B72', edgecolor='black')
            ax3.set_xlabel(f'{feat} (Original Scale)', fontsize=11, fontweight='bold')
            ax3.set_ylabel('Frequency', fontsize=11, fontweight='bold')
            ax3.set_title('Distribution in Training Data', fontsize=12, fontweight='bold')
            ax3.grid(alpha=0.3, axis='y')
            
            # Add statistics text
            stats_text = f'Mean: {orig_mean:.2f}\nStd: {orig_std:.2f}\nMin: {orig_min:.2f}\nMax: {orig_max:.2f}'
            ax3.text(0.98, 0.97, stats_text, transform=ax3.transAxes,
                    fontsize=9, verticalalignment='top', horizontalalignment='right',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
            
            # 4. ICE plot (bottom-right)
            ax4 = fig.add_subplot(gs[1, 1])
            
            # Sample 50 instances for ICE plot
            n_ice_samples = min(50, len(x_train_resampled))
            ice_indices = np.random.choice(len(x_train_resampled), n_ice_samples, replace=False)
            
            # Convert DataFrame to numpy array for indexing
            x_train_array = x_train_resampled.values if isinstance(x_train_resampled, pd.DataFrame) else x_train_resampled
            
            for idx in ice_indices:
                X_ice = np.tile(x_train_array[idx], (100, 1))
                X_ice[:, feat_idx] = orig_range
                ice_pred = model_for_pdp.predict_proba(X_ice)[:, 1]
                ax4.plot(orig_range, ice_pred, alpha=0.1, color='blue', linewidth=0.5)
            
            # Overlay PDP
            ax4.plot(orig_range, pd_values, linewidth=3, color='red', label='PDP (Average)')
            ax4.set_xlabel(f'{feat} (Original Scale)', fontsize=11, fontweight='bold')
            ax4.set_ylabel('Predicted Probability (Not Canceled)', fontsize=11, fontweight='bold')
            ax4.set_title('ICE Plot (Individual Conditional Expectation)', fontsize=12, fontweight='bold')
            ax4.grid(alpha=0.3)
            ax4.legend(loc='best')
            
            plt.suptitle(f'Comprehensive Analysis: {feat} (Overall Model)', fontsize=16, fontweight='bold', y=0.995)
            plt.savefig(f'pdp_overall_{feat}.png', dpi=300, bbox_inches='tight')
            print(f"✓ Saved: pdp_overall_{feat}.png")
            plt.close()
            
        else:
            # DISCRETE NUMERICAL FEATURES - 2-panel visualization
            fig, axes = plt.subplots(1, 2, figsize=(16, 6))
            
            if use_clustering:
                model_for_pdp = list(best_model_dict.values())[0]
            else:
                model_for_pdp = best_single_model
            
            # PDP plot
            display = PartialDependenceDisplay.from_estimator(
                model_for_pdp, x_train_resampled, features=[feat_idx],
                feature_names=all_feature_names, target=1, ax=axes[0], grid_resolution=50
            )
            axes[0].set_title(f'PDP: {feat}', fontsize=12, fontweight='bold')
            axes[0].set_xlabel(f'{feat} (Original Scale)', fontsize=11, fontweight='bold')
            axes[0].grid(alpha=0.3)
            
            # Distribution
            value_counts = pd.Series(orig_values).value_counts().sort_index()
            axes[1].bar(value_counts.index, value_counts.values, alpha=0.7, color='#A23B72', edgecolor='black')
            axes[1].set_xlabel(f'{feat}', fontsize=11, fontweight='bold')
            axes[1].set_ylabel('Frequency', fontsize=11, fontweight='bold')
            axes[1].set_title('Distribution in Training Data', fontsize=12, fontweight='bold')
            axes[1].grid(alpha=0.3, axis='y')
            
            # Add statistics text
            stats_text = f'Mean: {orig_mean:.2f}\nStd: {orig_std:.2f}\nMin: {int(orig_min)}\nMax: {int(orig_max)}'
            axes[1].text(0.98, 0.97, stats_text, transform=axes[1].transAxes,
                        fontsize=9, verticalalignment='top', horizontalalignment='right',
                        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
            
            plt.suptitle(f'Partial Dependence Analysis: {feat} (Overall Model)', fontsize=14, fontweight='bold')
            plt.tight_layout()
            plt.savefig(f'pdp_overall_{feat}.png', dpi=300, bbox_inches='tight')
            print(f"✓ Saved: pdp_overall_{feat}.png")
            plt.close()
    
    else:
        # CATEGORICAL (ENCODED) FEATURE
        print(f"  Type: Categorical (One-Hot Encoded)")
        
        # For categorical features, show simple PDP
        fig, ax = plt.subplots(1, 1, figsize=(10, 6))
        
        if use_clustering:
            model_for_pdp = list(best_model_dict.values())[0]
        else:
            model_for_pdp = best_single_model
        
        try:
            display = PartialDependenceDisplay.from_estimator(
                model_for_pdp, x_train_resampled, features=[feat_idx],
                feature_names=all_feature_names, target=1, ax=ax, grid_resolution=20
            )
            ax.set_title(f'Partial Dependence Plot: {feat} (Overall Model)', fontsize=13, fontweight='bold')
            ax.set_xlabel(f'{feat}', fontsize=11, fontweight='bold')
            ax.set_ylabel('Partial Dependence', fontsize=11, fontweight='bold')
            ax.grid(alpha=0.3)
            
            plt.tight_layout()
            plt.savefig(f'pdp_overall_{feat.replace("/", "_").replace(" ", "_")}.png', dpi=300, bbox_inches='tight')
            print(f"✓ Saved: pdp_overall_{feat.replace('/', '_').replace(' ', '_')}.png")
            plt.close()
        except Exception as e:
            print(f"⚠ Could not create PDP for {feat}: {str(e)}")
            plt.close()
            continue

# ============================================
# PARTIAL DEPENDENCE PLOTS PER CLUSTER (NEW SECTION)
# ============================================
if use_clustering:
    print("\n" + "="*80)
    print("PARTIAL DEPENDENCE PLOTS BY CLUSTER (Top 4 Features)")
    print("="*80)
    
    for cid in range(n_clusters):
        print(f"\n{'='*80}")
        print(f"CLUSTER {cid} - PARTIAL DEPENDENCE PLOTS")
        print(f"{'='*80}")
        
        # Get cluster-specific data
        cluster_mask = train_clusters == cid
        x_cluster = x_train_resampled[cluster_mask]
        y_cluster = y_train_resampled[cluster_mask]
        
        # Get cluster-specific model
        cluster_model = best_model_dict[cid]
        
        # Get top 4 features for this cluster
        cluster_top_4 = cluster_feature_importance[cid].head(4)['feature'].tolist()
        print(f"\nTop 4 features for Cluster {cid}: {cluster_top_4}")
        
        for feat in cluster_top_4:
            print(f"\n--- Processing PDP for Cluster {cid}: {feat} ---")
            
            feat_idx = all_feature_names.index(feat)
            is_numerical = feat in numerical_cols
            
            if is_numerical:
                # Get original values from the RESAMPLED training data (cluster-specific)
                # Extract from x_cluster which is already filtered by cluster_mask
                x_cluster_df = pd.DataFrame(x_cluster, columns=all_feature_names) if not isinstance(x_cluster, pd.DataFrame) else x_cluster
                orig_values_cluster = x_cluster_df[feat].values
                orig_min = orig_values_cluster.min()
                orig_max = orig_values_cluster.max()
                orig_mean = orig_values_cluster.mean()
                orig_std = orig_values_cluster.std()
                
                print(f"  Type: Numerical")
                print(f"  Min: {orig_min:.2f}, Max: {orig_max:.2f}, Mean: {orig_mean:.2f}, Std: {orig_std:.2f}")
                
                is_continuous = feat in continuous_numerical_cols
                
                if is_continuous:
                    # CONTINUOUS FEATURES - Comprehensive 4-panel visualization
                    fig = plt.figure(figsize=(16, 12))
                    gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)
                    
                    # 1. Standard PDP (top-left)
                    ax1 = fig.add_subplot(gs[0, 0])
                    display1 = PartialDependenceDisplay.from_estimator(
                        cluster_model, x_cluster, features=[feat_idx],
                        feature_names=all_feature_names, target=1, ax=ax1, grid_resolution=100
                    )
                    ax1.set_title(f'Partial Dependence Plot - Cluster {cid}', fontsize=12, fontweight='bold')
                    ax1.set_xlabel(f'{feat} (Original Scale)', fontsize=11, fontweight='bold')
                    ax1.grid(alpha=0.3)
                    
                    # 2. PDP with confidence intervals (top-right)
                    ax2 = fig.add_subplot(gs[0, 1])
                    
                    # Create range in original scale
                    orig_range = np.linspace(orig_min, orig_max, 100)
                    
                    # Create synthetic data for PD calculation
                    x_cluster_array = x_cluster.values if isinstance(x_cluster, pd.DataFrame) else x_cluster
                    X_synthetic = np.tile(x_cluster_array.mean(axis=0), (100, 1))
                    X_synthetic[:, feat_idx] = orig_range
                    
                    # Calculate predictions
                    pd_values = cluster_model.predict_proba(X_synthetic)[:, 1]
                    
                    # Center the PD values
                    pd_values_centered = pd_values - pd_values.mean()
                    
                    ax2.plot(orig_range, pd_values_centered, linewidth=2.5, color='#2E86AB', label='PDP')
                    ax2.fill_between(orig_range, pd_values_centered, alpha=0.3, color='#2E86AB')
                    ax2.set_xlabel(f'{feat} (Original Scale)', fontsize=11, fontweight='bold')
                    ax2.set_ylabel('Partial Dependence (Centered)', fontsize=11, fontweight='bold')
                    ax2.set_title(f'PDP with Shaded Area - Cluster {cid}', fontsize=12, fontweight='bold')
                    ax2.grid(alpha=0.3)
                    ax2.axhline(y=0, color='red', linestyle='--', alpha=0.5, linewidth=1)
                    
                    # Add percentile markers
                    percentiles = [10, 25, 50, 75, 90]
                    for p in percentiles:
                        val = np.percentile(orig_values_cluster, p)
                        ax2.axvline(x=val, color='gray', linestyle=':', alpha=0.3, linewidth=1)
                        ax2.text(val, ax2.get_ylim()[1]*0.95, f'P{p}', 
                                ha='center', va='top', fontsize=8, color='gray')
                    
                    # 3. Distribution of original values (bottom-left)
                    ax3 = fig.add_subplot(gs[1, 0])
                    ax3.hist(orig_values_cluster, bins=50, alpha=0.7, color='#A23B72', edgecolor='black')
                    ax3.set_xlabel(f'{feat} (Original Scale)', fontsize=11, fontweight='bold')
                    ax3.set_ylabel('Frequency', fontsize=11, fontweight='bold')
                    ax3.set_title(f'Distribution in Cluster {cid}', fontsize=12, fontweight='bold')
                    ax3.grid(alpha=0.3, axis='y')
                    
                    # Add statistics text
                    stats_text = f'Mean: {orig_mean:.2f}\nStd: {orig_std:.2f}\nMin: {orig_min:.2f}\nMax: {orig_max:.2f}'
                    ax3.text(0.98, 0.97, stats_text, transform=ax3.transAxes,
                            fontsize=9, verticalalignment='top', horizontalalignment='right',
                            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
                    
                    # 4. ICE plot (bottom-right)
                    ax4 = fig.add_subplot(gs[1, 1])
                    
                    # Sample 50 instances for ICE plot
                    n_ice_samples = min(50, len(x_cluster))
                    ice_indices = np.random.choice(len(x_cluster), n_ice_samples, replace=False)
                    
                    for idx in ice_indices:
                        X_ice = np.tile(x_cluster_array[idx], (100, 1))
                        X_ice[:, feat_idx] = orig_range
                        ice_pred = cluster_model.predict_proba(X_ice)[:, 1]
                        ax4.plot(orig_range, ice_pred, alpha=0.1, color='blue', linewidth=0.5)
                    
                    # Overlay PDP
                    ax4.plot(orig_range, pd_values, linewidth=3, color='red', label='PDP (Average)')
                    ax4.set_xlabel(f'{feat} (Original Scale)', fontsize=11, fontweight='bold')
                    ax4.set_ylabel('Predicted Probability (Not Canceled)', fontsize=11, fontweight='bold')
                    ax4.set_title(f'ICE Plot - Cluster {cid}', fontsize=12, fontweight='bold')
                    ax4.grid(alpha=0.3)
                    ax4.legend(loc='best')
                    
                    plt.suptitle(f'Comprehensive Analysis: {feat} - Cluster {cid}', fontsize=16, fontweight='bold', y=0.995)
                    plt.savefig(f'pdp_cluster{cid}_{feat}.png', dpi=300, bbox_inches='tight')
                    print(f"✓ Saved: pdp_cluster{cid}_{feat}.png")
                    plt.close()
                    
                else:
                    # DISCRETE NUMERICAL FEATURES - 2-panel visualization
                    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
                    
                    # PDP plot
                    display = PartialDependenceDisplay.from_estimator(
                        cluster_model, x_cluster, features=[feat_idx],
                        feature_names=all_feature_names, target=1, ax=axes[0], grid_resolution=50
                    )
                    axes[0].set_title(f'PDP: {feat} - Cluster {cid}', fontsize=12, fontweight='bold')
                    axes[0].set_xlabel(f'{feat} (Original Scale)', fontsize=11, fontweight='bold')
                    axes[0].grid(alpha=0.3)
                    
                    # Distribution
                    value_counts = pd.Series(orig_values_cluster).value_counts().sort_index()
                    axes[1].bar(value_counts.index, value_counts.values, alpha=0.7, color='#A23B72', edgecolor='black')
                    axes[1].set_xlabel(f'{feat}', fontsize=11, fontweight='bold')
                    axes[1].set_ylabel('Frequency', fontsize=11, fontweight='bold')
                    axes[1].set_title(f'Distribution in Cluster {cid}', fontsize=12, fontweight='bold')
                    axes[1].grid(alpha=0.3, axis='y')
                    
                    # Add statistics text
                    stats_text = f'Mean: {orig_mean:.2f}\nStd: {orig_std:.2f}\nMin: {int(orig_min)}\nMax: {int(orig_max)}'
                    axes[1].text(0.98, 0.97, stats_text, transform=axes[1].transAxes,
                            fontsize=9, verticalalignment='top', horizontalalignment='right',
                            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
                    
                    plt.suptitle(f'Partial Dependence Analysis: {feat} - Cluster {cid}', fontsize=14, fontweight='bold')
                    plt.tight_layout()
                    plt.savefig(f'pdp_cluster{cid}_{feat}.png', dpi=300, bbox_inches='tight')
                    print(f"✓ Saved: pdp_cluster{cid}_{feat}.png")
                    plt.close()
            
            else:
                # CATEGORICAL (ENCODED) FEATURE
                print(f"  Type: Categorical (One-Hot Encoded)")
                
                # For categorical features, show simple PDP
                fig, ax = plt.subplots(1, 1, figsize=(10, 6))
                
                try:
                    display = PartialDependenceDisplay.from_estimator(
                        cluster_model, x_cluster, features=[feat_idx],
                        feature_names=all_feature_names, target=1, ax=ax, grid_resolution=20
                    )
                    ax.set_title(f'Partial Dependence Plot: {feat} - Cluster {cid}', fontsize=13, fontweight='bold')
                    ax.set_xlabel(f'{feat}', fontsize=11, fontweight='bold')
                    ax.set_ylabel('Partial Dependence', fontsize=11, fontweight='bold')
                    ax.grid(alpha=0.3)
                    
                    plt.tight_layout()
                    plt.savefig(f'pdp_cluster{cid}_{feat.replace("/", "_").replace(" ", "_")}.png', dpi=300, bbox_inches='tight')
                    print(f"✓ Saved: pdp_cluster{cid}_{feat.replace('/', '_').replace(' ', '_')}.png")
                    plt.close()
                except Exception as e:
                    print(f"⚠ Could not create PDP for {feat} in Cluster {cid}: {str(e)}")
                    plt.close()
                    continue

print("\n" + "="*80)
print("ANALYSIS COMPLETE!")
print("="*80)
print("\nGenerated Files:")
print("  • model_metrics_comprehensive.csv")
print("  • model_comparison_metrics.png")
print("  • feature_importance_overall.png")
if use_clustering:
    print(f"  • feature_importance_cluster_0.png")
    print(f"  • feature_importance_cluster_1.png")
    print(f"  • feature_importance_clusters_comparison.png")
print("  • confusion_matrix_*.png (4 files)")
print("  • pdp_overall_*.png (Top 4 features - overall model)")
if use_clustering:
    print("  • pdp_cluster0_*.png (Top 4 features - cluster 0)")
    print("  • pdp_cluster1_*.png (Top 4 features - cluster 1)")
print("\n" + "="*80)
print("METRICS LEGEND:")
print("="*80)
print("• Accuracy:  Overall correctness (TP+TN)/(TP+TN+FP+FN)")
print("• Precision: Of predicted cancellations, how many were correct? TP/(TP+FP)")
print("• Recall:    Of actual cancellations, how many did we catch? TP/(TP+FN)")
print("• F1-Score:  Harmonic mean of Precision and Recall (balanced metric)")
print("\nNote: All metrics calculated for 'Canceled' as positive class (class 0)")
print("="*80)

HOTEL BOOKING CANCELLATION PREDICTION & RECOMMENDATION SYSTEM

Original shape: (36275, 19)
Correcting 37 invalid dates (2018-02-29 → 2018-02-28)
After preprocessing: 36275 records

Train: (29020, 21), Test: (7255, 21)

⚠️ Using ORIGINAL data (No scaling applied)
Feature columns: 30
  Categorical (encoded): 15
  Numerical (original): 15

Target encoding: {'Canceled': 0, 'Not_Canceled': 1}
  0 = Canceled, 1 = Not_Canceled

After SMOTE: 39024 samples
Class balance: {0: 19512, 1: 19512}

APPROACH 1: STANDARD MODELS (No Clustering)

--- Random Forest (Standard) ---
Train - Acc: 0.9418, Prec: 0.9498, Rec: 0.9330, F1: 0.9413
Test  - Acc: 0.8976, Prec: 0.8644, Rec: 0.8153, F1: 0.8391
✓ Saved: confusion_matrix_rf_standard.png

--- XGBoost (Standard) ---
Train - Acc: 0.9078, Prec: 0.9184, Rec: 0.8951, F1: 0.9066
Test  - Acc: 0.8852, Prec: 0.8374, Rec: 0.8061, F1: 0.8214
✓ Saved: confusion_matrix_xgb_standard.png

APPROACH 2: CLUSTER-BASED MODELS

Cluster Distribution:
Train: {0: 27763, 1: 11261}